In [2]:
# 导入 pyope 核心模块
from pyope import (
    BasisOperator,           # 基本算符
    DerivativeOperator,      # 导数算符
    NormalOrderedOperator,   # 正规序算符
    d, dn,                   # 导数函数
    One, Zero, Delta,        # 常数算符
    is_local_operator,       # 判断函数

)

# 导入 sympy 用于符号计算
import sympy as sp
from sympy import symbols, simplify

# 启用 LaTeX 渲染（在 Jupyter 中显示数学公式）
sp.init_printing(use_latex='mathjax')

print("PyOPE 模块导入成功！")

PyOPE 模块导入成功！


## 9. 新实现的核心功能：OPE 计算

以下是刚刚实现的核心 OPE 计算功能的演示。

### 9.1 导入 OPE 相关模块

In [3]:
# 导入 OPE 计算相关的新功能
from pyope import (
    OPE,           # OPE 计算器（支持定义和计算）
    NO,            # 正规序函数
    bracket,       # bracket 函数
    MakeOPE,       # 创建 OPEData
    Bosonic,       # 声明玻色算符
    Fermionic,     # 声明费米算符
    OPEData,       # OPE 数据类
)

print("✓ OPE 计算模块导入成功！")

✓ OPE 计算模块导入成功！


### 9.2 声明算符类型

在定义 OPE 之前，我们需要声明算符是玻色子还是费米子。

In [4]:
# 创建新的算符用于 OPE 演示
T_ope = BasisOperator("T", bosonic=True, conformal_weight=2)
J = BasisOperator("J", bosonic=True, conformal_weight=1)
psi_ope = BasisOperator("psi", bosonic=False, conformal_weight=1.5)

# 声明算符类型（注册到全局注册表）
Bosonic(T_ope, J)
Fermionic(psi_ope)

print("✓ 算符类型声明完成")
print(f"  - T_ope: 玻色算符")
print(f"  - J: 玻色算符")
print(f"  - psi_ope: 费米算符")

✓ 算符类型声明完成
  - T_ope: 玻色算符
  - J: 玻色算符
  - psi_ope: 费米算符


### 9.3 定义 OPE

使用 `OPE[A, B] = ...` 语法定义基本算符的 OPE。

In [5]:
# 定义 Virasoro 代数的 OPE: T(z)T(w)
# T(z)T(w) = (c/2)/(z-w)^4 + 2T(w)/(z-w)^2 + ∂T(w)/(z-w)

c = sp.Symbol('c')  # 中心荷

# 使用 MakeOPE 或 OPE.make 创建 OPEData
# 列表按从高阶到低阶排列：[pole_4, pole_3, pole_2, pole_1]
OPE[T_ope, T_ope] = OPE.make([
    c/2 * One,    # 4阶极点
    0,            # 3阶极点（无）
    2 * T_ope,    # 2阶极点
    d(T_ope)      # 1阶极点
])

print("✓ 定义了 Virasoro 代数的 OPE:")
print(f"  OPE[T, T] = (c/2)/(z-w)^4 + 2T/(z-w)^2 + ∂T/(z-w)")

✓ 定义了 Virasoro 代数的 OPE:
  OPE[T, T] = (c/2)/(z-w)^4 + 2T/(z-w)^2 + ∂T/(z-w)


In [8]:
# 定义更简单的 OPE: J(z)J(w)
# J(z)J(w) = k/(z-w)^2

k = sp.Symbol('k')  # 水平

OPE[J, J] = OPE.make([
    k * One,      # 2阶极点
    0             # 1阶极点（无）
])

print("✓ 定义了电流代数的 OPE:")
print(f"  OPE[J, J] = k/(z-w)^2")

✓ 定义了电流代数的 OPE:
  OPE[J, J] = k/(z-w)^2


### 9.4 计算 OPE

使用 `OPE(A, B)` 计算两个算符的 OPE。

In [6]:
# 计算 T(z)T(w) 的 OPE
ope_TT = OPE(T_ope, T_ope)

print("计算 OPE(T, T):")
print(f"  类型: {type(ope_TT).__name__}")
print(f"  最高极点: {ope_TT.max_pole}")
print(f"  极点字典: {ope_TT.poles}")
print()

# 查看各个极点的系数
print("各阶极点的系数:")
for n in range(ope_TT.max_pole, 0, -1):
    coeff = ope_TT.pole(n)
    if coeff != 0:
        print(f"  pole({n}): {coeff}")

计算 OPE(T, T):
  类型: OPEData
  最高极点: 4
  极点字典: {4: One*c/2, 2: 2*T, 1: d(T)}

各阶极点的系数:
  pole(4): One*c/2
  pole(2): 2*T
  pole(1): ∂T


In [9]:
# 计算 J(z)J(w) 的 OPE
ope_JJ = OPE(J, J)

print("计算 OPE(J, J):")
print(f"  最高极点: {ope_JJ.max_pole}")
print(f"  pole(2): {ope_JJ.pole(2)}")
print(f"  pole(1): {ope_JJ.pole(1)}")

计算 OPE(J, J):
  最高极点: 2
  pole(2): One*k
  pole(1): 0


### 9.5 OPE 的线性性

OPE 计算自动支持线性性：`OPE(A, B+C) = OPE(A,B) + OPE(A,C)`

In [10]:
# 测试线性性：OPE(T, J+J) = OPE(T,J) + OPE(T,J)
# 注意：T 和 J 的 OPE 未定义，所以返回零

ope_sum = OPE(T_ope, J + J)
print(f"OPE(T, J+J) = {ope_sum}")
print(f"是否为零: {ope_sum.is_zero()}")
print()

# 测试标量乘法：OPE(2*T, T) = 2*OPE(T,T)
ope_scaled = OPE(2*T_ope, T_ope)
print(f"OPE(2*T, T) 的最高极点: {ope_scaled.max_pole}")
print(f"pole(4) = {ope_scaled.pole(4)}")  # 应该是 2*(c/2) = c
print(f"pole(2) = {ope_scaled.pole(2)}")  # 应该是 2*2*T = 4*T

OPE(T, J+J) = 0
是否为零: True

OPE(2*T, T) 的最高极点: 4
pole(4) = One*c
pole(2) = 4*T


### 9.6 使用 bracket 函数

`bracket(A, B, n)` 提取 OPE 的第 n 阶极点系数，即 {AB}_n。

In [11]:
# 使用 bracket 提取特定极点
bracket_4 = bracket(T_ope, T_ope, 4)
bracket_2 = bracket(T_ope, T_ope, 2)
bracket_1 = bracket(T_ope, T_ope, 1)
bracket_0 = bracket(T_ope, T_ope, 0)

print("使用 bracket 函数提取 OPE 的极点系数:")
print(f"  {{TT}}_4 = {bracket_4}")
print(f"  {{TT}}_2 = {bracket_2}")
print(f"  {{TT}}_1 = {bracket_1}")
print(f"  {{TT}}_0 = {bracket_0}")
print()
print("注意：bracket(A, B, 0) = NO(A, B) 是正规序乘积")

使用 bracket 函数提取 OPE 的极点系数:
  {TT}_4 = One*c/2
  {TT}_2 = 2*T
  {TT}_1 = ∂T
  {TT}_0 = 0

注意：bracket(A, B, 0) = NO(A, B) 是正规序乘积


### 9.7 使用 NO 函数

`NO(A, B)` 计算正规序乘积，等价于 `bracket(A, B, 0)`。

In [12]:
# 使用 NO 函数创建正规序乘积
no_TJ = NO(T_ope, J)
print(f"NO(T, J) = {no_TJ}")
print(f"类型: {type(no_TJ).__name__}")
print()

# NO 函数支持线性性
no_sum = NO(T_ope + J, J)
print(f"NO(T+J, J) = {no_sum}")
print()

# NO 函数支持标量乘法
no_scaled = NO(2*T_ope, J)
print(f"NO(2*T, J) = {no_scaled}")
print()

# NO 与零算符
no_zero = NO(T_ope, 0)
print(f"NO(T, 0) = {no_zero}")

NO(T, J) = NO(T,J)
类型: NormalOrderedOperator

NO(T+J, J) = NO(J,J) + NO(T,J)

NO(2*T, J) = 2*NO(T,J)

NO(T, 0) = 0


### 9.8 对易子和反对易子

`bracket` 函数还可以计算对易子和反对易子。

In [13]:
# 玻色算符的对易子: [A, B] = NO(A,B) - NO(B,A)
commutator_TJ = bracket(T_ope, J, anticommutator=False)
print(f"对易子 [T, J] = {commutator_TJ}")
print()

# 费米算符的反对易子: {A, B} = NO(A,B) + NO(B,A)
anticommutator_psi = bracket(psi_ope, psi_ope, anticommutator=True)
print(f"反对易子 {{psi, psi}} = {anticommutator_psi}")

对易子 [T, J] = -NO(J,T) + NO(T,J)

反对易子 {psi, psi} = 2*NO(psi,psi)


### 9.9 OPEData 的算术运算

OPEData 对象支持加法、减法和标量乘法。

In [17]:
# OPEData 的加法
ope1 = OPE(T_ope, T_ope)
ope2 = OPE(J, J)
ope_sum = ope1 + ope2

print("OPEData 加法:")
print(f"  OPE(T,T) + OPE(J,J) 的最高极点: {ope_sum.max_pole}")
print(f"  pole(4): {ope_sum.pole(4)}")
print(f"  pole(2): {ope_sum.pole(2)}")
print()

# OPEData 的标量乘法
ope_scaled = 3 * ope1
print("OPEData 标量乘法:")
print(f"  3 * OPE(T,T) 的 pole(4): {ope_scaled.pole(4)}")
print(f"  3 * OPE(T,T) 的 pole(2): {ope_scaled.pole(2)}")
print()

# OPEData 的减法
ope_diff = ope1 - ope1
print("OPEData 减法:")
print(f"  OPE(T,T) - OPE(T,T) 是否为零: {ope_diff.is_zero()}")

OPEData 加法:
  OPE(T,T) + OPE(J,J) 的最高极点: 4
  pole(4): One*c/2
  pole(2): 2*T + One*k

OPEData 标量乘法:
  3 * OPE(T,T) 的 pole(4): 3*One*c/2
  3 * OPE(T,T) 的 pole(2): 6*T

OPEData 减法:
  OPE(T,T) - OPE(T,T) 是否为零: True


### 9.10 复合算符的 OPE（完整实现）

PyOPE 现在完整支持复合算符的 OPE 计算，包括：
- 右侧复合：`OPE(A, NO(B,C))`
- 左侧复合：`OPE(NO(A,B), C)` ✨ **新功能**

这些计算使用完整的 Jacobi 恒等式实现。

In [34]:
OPE[T_ope, J] = OPE.make([J, d(J)])

In [35]:
OPE(T_ope, d(J))

OPEData({2: ∂J, 1: J + ∂^2J})

In [18]:
# 创建嵌套的正规序算符
no_JJ = NO(J, J)
no_nested = NO(no_JJ, J)

print(f"嵌套正规序: {no_nested}")
print(f"类型: {type(no_nested).__name__}")
print()

# 计算嵌套正规序算符的 OPE
ope_nested = OPE(no_nested, J)
print(f"OPE(NO(NO(J,J), J), J) 最高极点: {ope_nested.max_pole}")
print()
print("各阶极点:")
for q in range(min(ope_nested.max_pole, 3), 0, -1):
    pole_q = ope_nested.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

嵌套正规序: NO(NO(J,J),J)
类型: NormalOrderedOperator

OPE(NO(NO(J,J), J), J) 最高极点: 2

各阶极点:
  pole(2): 3*NO(J,J)*k
  pole(1): ∂NO(J,J)*k + 2*NO(J,∂J)*k + 2*NO(∂J,J)*k


### 9.11 嵌套正规序算符

PyOPE 支持嵌套的正规序算符，例如 `NO(NO(A,B), C)`。

In [ ]:
# 示例 3: 比较左侧和右侧复合算符的 OPE
# OPE(NO(J,J), J) vs OPE(J, NO(J,J))

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)
ope_right = OPE(J, no_JJ)

print("示例 3: 左侧 vs 右侧复合算符")
print(f"OPE(NO(J,J), J) 最高极点: {ope_left.max_pole}")
print(f"OPE(J, NO(J,J)) 最高极点: {ope_right.max_pole}")
print()
print("左侧复合 - pole(2):", ope_left.pole(2))
print("右侧复合 - pole(2):", ope_right.pole(2))

In [ ]:
# 示例 2: 计算 OPE(NO(T,J), J)
# 混合算符的左侧复合 OPE

no_TJ = NO(T_ope, J)
ope_mixed = OPE(no_TJ, J)

print("示例 2: OPE(NO(T,J), J)")
print(f"最高极点: {ope_mixed.max_pole}")
print()
print("各阶极点:")
for q in range(ope_mixed.max_pole, 0, -1):
    pole_q = ope_mixed.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 示例 1: 计算 OPE(NO(J,J), J)
# 这是左侧复合算符的 OPE

no_JJ = NO(J, J)
ope_left = OPE(no_JJ, J)

print("示例 1: OPE(NO(J,J), J)")
print(f"最高极点: {ope_left.max_pole}")
print()
print("各阶极点:")
for q in range(ope_left.max_pole, 0, -1):
    pole_q = ope_left.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

In [ ]:
# 定义 T 和 J 的混合 OPE
OPE[T_ope, J] = OPE.make([
    J,          # pole 2
    d(J)        # pole 1
])

print("✓ 定义了混合 OPE: T(z)J(w) = J/(z-w)^2 + ∂J/(z-w)")